In [1]:
import pandas as pd
from pathlib import Path
import warnings

# Suppress openpyxl data validation warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

# Define folder path and find all Excel files
folder_path = Path("daily-report/02-2026/")
excel_files = [f for f in folder_path.glob("*.xlsx") if not f.name.startswith("~$")]

# Column mapping
column_mapping = {
    "Report Date": "Report Date",
    "Region": "Region",
    "Zone": "Zone",
    "Unit Name": "Rig Name",
    "Well Name/ Location": "Well Name_1",
    "Job Type": "Well Type",
    "Summary": "Summary Report",
    "Next Plan": "Next Plan"
}

# Process each file and store in list
dfs = []
for file_path in excel_files:
    # Load data
    df = pd.read_excel(
        file_path,
        sheet_name="Bor Report Region 02",
        header=5
    )
    
    # Filter and rename columns
    df = df[list(column_mapping.keys())].rename(columns=column_mapping)
    df = df[df["Zone"].isin(["Zone_05", "Zone_06"])]
    
    # Replace values
    df["Well Type"] = df["Well Type"].replace({"BOR EKS": "Exploration", "BOR DEV": "Development"})
    df["Region"] = df["Region"].replace({"Reg_02": "Region 2"})
    df["Zone"] = df["Zone"].replace({"Zone_05": "Zone 5", "Zone_06": "Zone 6"})
    
    # Add APH column based on Zone
    df["APH"] = df["Zone"].map({"Zone 5": "ONWJ", "Zone 6": "OSES"})
    
    # Duplicate Well Name column
    df["Well Name_2"] = df["Well Name_1"]
    
    # Add Operation Date column (Report Date minus 1 day)
    df["Operation Date"] = pd.to_datetime(df["Report Date"]) - pd.Timedelta(days=1)
    
    # Replace 'PVD-I' with 'PVD-II' in 'Rig Name' column
    df["Rig Name"] = df["Rig Name"].replace("PVD-I", "PVD-II")
    
    # Add columns with default values
    df["Flag"] = "INC"
    df["Location"] = "Offshore"
    df["Spud Date"] = ""
    df["Release Date"] = ""
    df["Status"] = ""
    df["Status Code [1]"] = ""
    df["Status Code [2]"] = ""
    df["Current Status"] = ""
    
    # Reorder columns
    column_order = [
        "Flag", "Region", "Zone", "APH", "Rig Name", "Well Name_1", "Well Name_2",
        "Well Type", "Location", "Spud Date", "Release Date", "Status",
        "Status Code [1]", "Status Code [2]", "Summary Report", "Current Status",
        "Next Plan", "Report Date", "Operation Date"
    ]
    df = df.reindex(columns=column_order)
    
    dfs.append(df)

# Merge all dataframes
df = pd.concat(dfs, ignore_index=True)

# Sort by Zone and Rig Name
df = df.sort_values(by=["Zone", "Rig Name"])

In [ ]:
# Add date picker widget and display dataframe

import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import timedelta

# Create date picker widget
date_picker = widgets.DatePicker(
    description='Pick a date:',
    disabled=False
)

# Create increment/decrement buttons
btn_decrement = widgets.Button(description='◀ Previous Day', button_style='info')
btn_increment = widgets.Button(description='Next Day ▶', button_style='info')

# Function to decrement date by one day
def decrement_date(btn):
    if date_picker.value is not None:
        date_picker.value = date_picker.value - timedelta(days=1)

# Function to increment date by one day
def increment_date(btn):
    if date_picker.value is not None:
        date_picker.value = date_picker.value + timedelta(days=1)

# Link buttons to functions
btn_decrement.on_click(decrement_date)
btn_increment.on_click(increment_date)

# Create horizontal box layout for widgets
widget_box = widgets.HBox([btn_decrement, date_picker, btn_increment])

# Function to filter and display DataFrame on date select
def show_df_by_date(change):
    # Clear previous output (except widget)
    clear_output(wait=True)
    display(widget_box)
    # Get selected date from widget
    selected_date = date_picker.value
    # Filter DataFrame by selected date
    if selected_date is not None:
        # Convert datetime to date for comparison
        filtered_df = df[df["Report Date"].dt.date == selected_date]
    else:
        filtered_df = df
    display(filtered_df)
    # Copy to clipboard
    filtered_df.to_clipboard(header=False, index=False)

# Show DataFrame by default (filter by APH only)
clear_output(wait=True)
display(widget_box)
display(df)

# Run display on date change
date_picker.observe(show_df_by_date, names='value')

,Flag,Region,Zone,APH,Rig Name,Well Name_1,Well Name_2,Well Type,Location,Spud Date,Release Date,Status,Status Code [1],Status Code [2],Summary Report,Current Status,Next Plan,Report Date,Operation Date
3,INC,Region 2,Zone 5,ONWJ,PVD-II,LFE-1,LFE-1,Exploration,Offshore,,,,,,"Cut 9⅝"" at 362 Ft. Pull & recovered 9⅝"" casing...",,"Cont. cut & pull 9⅝"" casing. R/U 9⅝"" Nabors ha...",2026-02-02,2026-02-01
4,INC,Region 2,Zone 5,ONWJ,PVD-III,OOA-2ST2,OOA-2ST2,Development,Offshore,,,,,,"Ream Down with 7"" Gauge Run / WBCO Assy from 4...",,"Circulate and Condition Mud. ⁠POOH 7"" Gauge Ru...",2026-02-02,2026-02-01
5,INC,Region 2,Zone 6,OSES,Jack Up Emerald Driller,Yvonne A-11 (Slot Recovery).,Yvonne A-11 (Slot Recovery).,Development,Offshore,,,,,,POOH 12-1/4” RSS/MLWD BHA from 3271 ftMD to 31...,,"Continue directional drill 12-1/4"" Hole.",2026-02-02,2026-02-01
